In [ ]:
from utils import load_stop_times_df
import pandas as pd
from datetime import datetime
import numpy as np
from glob import glob
from tqdm import tqdm
import os
from joypy import joyplot
import matplotlib.pyplot as plt

In [126]:
result_list = []
all_data = []

def get_data(city):

    df = load_stop_times_df(os.path.basename(city))

    df["arrival_time"] = pd.to_datetime(df.arrival_time, errors = "coerce", format = "%H:%M:%S")
    df["departure_time"] = pd.to_datetime(df.departure_time, errors = "coerce", format = "%H:%M:%S")

    resdict = {}

    for i in range(0, 24):

        # year, month, day, uhrzeit
        time_format = '%m/%d/%Y %H:%M:%S'

        # startuhrzeit
        hour = datetime(1900, 1, 1, i,0,0 )

        # enduhrzeit, da 24 Uhr gleich 0 sein muss, 
        # Datensatz ist komisch, deswegen zwischen 23 und 0 Uhr keine Fahrten
        if i != 24:
            hour_later = hour + pd.Timedelta(hours = 1)
            #print(hour_later)
        else:
            hour_later = datetime.strptime("01/01/1900 " + "00:00:00", time_format)

        number = len(df.query("@hour <= arrival_time <= @hour_later"))
        
        # print("von", hour, "bis", hour_later, number)

        resdict[i] = number

    # we dont have values between 23 and 24, datetimes are hard, this aproximates it and makes the graph prettier :)
    resdict[24] = resdict[0]

    reslist = []
    for i in resdict.values():
        reslist.append(i)

    x = list(resdict.keys())
    y = list(resdict.values())

    # peak time 
    peakx = np.argmax(y)

    result_list.append([city, peakx])

    # append all data to a dict
    all_data.append([city, x, y])
    return

all_city_paths = glob("data/*")
all_city_paths.pop(0)

for city_name in tqdm(all_city_paths):
    try:
        get_data(city_name)
    except Exception as e:
        pass

fulldf = pd.DataFrame(all_data, columns = ["name", "x","y"])

res_df = pd.DataFrame(result_list, columns = ["name","time"])
print(res_df.head())

res_df.to_csv("peakusage.csv")
fulldf.to_csv("fullusagedata.csv")

  2%|▏         | 9/400 [00:01<00:38, 10.14it/s]e:\TSA\Datascience2\utils.py:32: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep = ",", header = 0)
C:\Users\Lauris\AppData\Local\Temp\ipykernel_3312\616493329.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["arrival_time"] = pd.to_datetime(df.arrival_time, errors = "coerce", format = "%H:%M:%S")
C:\Users\Lauris\AppData\Local\Temp\ipykernel_3312\616493329.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

                         name  time
0  data\US_Alabama_Birmingham     9
1  data\US_Alabama_Huntsville    17
2  data\US_Alabama_Montgomery     8
3    data\US_Alaska_Anchorage    17
4       data\US_Alaska_Juneau    16
